# Imports

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.model_selection import StratifiedKFold, GridSearchCV, LeaveOneOut

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [2]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 13                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data24h"
DEPRESJON_PREFIX = "manual_depresjon24h"
HYPERAKTIV_PREFIX = "manual_hyperaktiv24h"
PSYKOSE_PREFIX = "manual_psykose24h"

# Manual feature extraction

## Helper functions

In [3]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/13),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/13),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [4]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [5]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [6]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 13hs
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "day", corresponding to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("day", full_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [7]:
dataset = Dataset(dirpath=os.path.join("data_day", "depresjon"))
condition = dataset.condition
control = dataset.control

In [8]:
condition[0]

,timestamp,date,activity
0,2003-05-08 08:00:00,2003-05-08,3
1,2003-05-08 08:01:00,2003-05-08,0
2,2003-05-08 08:02:00,2003-05-08,0
3,2003-05-08 08:03:00,2003-05-08,0
4,2003-05-08 08:04:00,2003-05-08,0
...,...,...,...
775,2003-05-08 20:55:00,2003-05-08,52
776,2003-05-08 20:56:00,2003-05-08,20
777,2003-05-08 20:57:00,2003-05-08,5
778,2003-05-08 20:58:00,2003-05-08,0


In [9]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [10]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [11]:
condition_df

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,75.916664,395.649994,265.802551,279.883331,12065.484375,-1.171582,-0.431376,0.397038,187.633347,271.260620,...,2.998658e+05,3.341706e+05,4.923075e+10,-1.275133,0.156652,0.739931,3.461079e+05,2.998658e+05,2.332665,0.502459
1,7.733333,432.483337,197.839752,192.399994,15148.465820,-0.583989,0.079738,0.597709,115.433334,193.790909,...,3.570634e+05,1.394110e+05,1.262747e+11,-0.791910,0.868935,0.995206,4.617640e+05,3.570634e+05,2.123986,0.543610
2,60.083332,447.583344,214.744858,206.699997,12191.085938,-0.223079,0.590566,0.493989,74.533340,207.637894,...,1.052825e+05,1.185609e+05,3.691578e+09,-1.002532,0.076838,0.577098,8.095201e+04,1.052825e+05,2.537145,0.777841
3,57.200001,299.966675,162.461533,152.283340,5171.954590,-0.538682,0.400807,0.425300,72.949997,159.530289,...,1.598933e+05,1.536467e+05,8.813271e+09,-1.191926,0.013451,0.587135,1.376897e+05,1.598933e+05,2.515386,0.724468
4,16.900000,1137.300049,562.648743,368.250000,138058.640625,-1.400442,0.168262,0.634473,574.883362,560.021240,...,2.241901e+06,1.525539e+06,1.605035e+12,-1.177838,0.646454,0.565101,1.875858e+06,2.241901e+06,2.587361,0.854907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,0.000000,896.299988,270.273071,225.800003,90396.242188,-0.214286,1.014289,1.068787,279.916669,237.931824,...,2.767844e+06,1.901925e+06,4.884883e+12,-1.300258,0.616732,0.798519,3.505501e+06,2.767844e+06,2.353370,0.689271
355,0.000000,12.933333,1.742308,0.000000,13.195168,5.184593,2.479002,2.003095,2.150000,0.883333,...,7.306306e+01,2.426324e+01,5.333601e+03,-0.984258,0.706903,0.999568,1.056915e+02,7.306306e+01,2.069320,0.454013
356,0.000000,7.000000,0.617949,0.000000,3.733586,7.753702,3.096233,3.004205,0.050000,0.093939,...,2.073447e+02,2.293404e+02,6.252569e+03,0.422520,-1.275969,0.381361,6.715626e+01,2.073446e+02,2.666549,0.855857
357,0.000000,8.800000,1.008974,0.000000,6.390724,5.248307,2.568532,2.407209,0.250000,0.392424,...,2.454599e+01,2.050219e+01,2.678947e+02,1.075974,1.531070,0.666809,1.094104e+01,2.454599e+01,2.540185,0.840861


In [12]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [13]:
# to get ensemble:
import os

def sort_key(filename):
    parts = filename.split("_")
    # Wydobycie numerów jako liczb całkowitych
    condition_number = int(parts[1])  # Druga część np. "1" z "condition_1"
    segment_number = int(parts[3].split(".")[0])  # Trzecia część np. "1" z "segmen1.csv"
    return (condition_number, segment_number)


In [14]:
# Removing NaNs
folder_control = './data_day/depresjon/control'
folder_condition = './data_day/depresjon/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["day"].index

mask = datasets["day"].notna().all(axis=1)
datasets["day"] = datasets["day"][mask]
y = y[mask]
z_depresjon = z[mask]

## Psykose

In [15]:
dataset = Dataset(dirpath=os.path.join("data_day", "psykose"))
condition = dataset.condition
control = dataset.control

In [16]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [17]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [18]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)
#datasets["full_24h"].dropna(inplace=True)

In [19]:
# Removing NaNs
folder_control = './data_day/psykose/control'
folder_condition = './data_day/psykose/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["day"].index
mask = datasets["day"].notna().all(axis=1)
datasets["day"] = datasets["day"][mask]
y = y[mask]
z_psykose = z[mask]

Hyperactive

In [20]:
dataset = Dataset(dirpath=os.path.join("data_day", "hyperaktiv"))
condition = dataset.condition
control = dataset.control

In [21]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [22]:
for part, df in datasets.items():
    filename = f"{HYPERAKTIV_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [23]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"hyperaktiv_y.csv")
y.to_csv(filepath, header=False, index=False)

In [24]:
datasets["day"].dropna(inplace=True)
datasets["day"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,5.450000,147.433334,62.847435,43.950001,2284.001221,-0.914054,0.728419,0.730600,72.199997,60.375755,...,38759.402344,21145.060547,2.928774e+09,1.929085,1.929875,1.396258,14845.820312,38759.402344,1.827267,0.496738
1,0.000000,9.133333,2.510257,2.450000,7.301598,0.679430,1.063763,1.034212,3.283333,2.136364,...,78.993980,91.679405,1.346396e+03,-1.501138,-0.275953,0.464507,61.959623,78.993980,2.633243,0.863299
2,0.000000,82.599998,15.712819,3.916667,611.214355,2.285171,1.848395,1.511686,12.950000,11.060606,...,28950.376953,25248.203125,2.239280e+08,-0.721994,0.360955,0.516892,18206.178711,28950.378906,2.605718,0.845312
3,2.533333,403.383331,134.980759,91.349998,16141.065430,-0.447485,0.889049,0.904301,162.066673,122.621216,...,150422.703125,97665.960938,1.761064e+10,-0.007812,1.150318,0.882214,127027.554688,150422.703125,2.306046,0.677691
4,7.500000,62.933334,32.089745,30.549999,320.231171,-0.919846,0.394668,0.535778,21.216667,31.521210,...,5731.346191,2009.573608,3.918533e+07,0.205687,1.285825,1.092207,6139.816345,5731.346191,2.076416,0.573473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,22.333334,550.450012,196.321777,135.466660,26017.556641,0.155375,1.174797,0.789376,164.333351,179.945450,...,174145.281250,81364.820312,1.823809e+10,-0.825157,0.761090,0.775493,190299.281250,174145.265625,2.393930,0.729809
510,44.416668,393.549988,152.557693,100.116669,12873.285156,-0.312746,0.961288,0.714545,141.866661,140.480301,...,91368.007812,53296.035156,6.254545e+09,0.120749,1.158702,0.865573,84590.829102,91368.000000,2.324360,0.691359
511,70.900002,528.700012,210.732040,194.216660,19874.033203,-0.017442,0.967725,0.642734,195.466667,194.537857,...,266050.906250,208450.796875,1.799971e+10,0.540082,1.227805,0.504276,105362.554688,266050.937500,2.642954,0.894867
512,52.366665,272.850006,116.820511,100.366669,4457.985840,0.607930,1.324658,0.549122,55.699997,108.495453,...,118946.867188,55876.003906,1.687564e+10,-0.832786,0.962896,1.092137,135598.316406,118946.867188,2.000861,0.389556


# Classification

## Classifiers, parameters, constants

In [25]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    ),
    "LGBM": LGBMClassifier(
        n_estimators=200,
        verbosity=-1,
        random_state=0
    ),
    "XGB": XGBClassifier(
        n_estimators=200,
        random_state=0 
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": ["balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
        "max_iter": [1000]
    },
    "SVM": {
        "C": [1, 10],
        "gamma": ["scale"],
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    },
    "LGBM": {
        "num_leaves": [31, 50],
        "min_child_samples": [10, 20],
        "class_weight": [None, "balanced"]
    },
    "XGB": {
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 6],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "scale_pos_weight": [1, 10]
    }
}

## Classification

# Classification - Depresjon

In [26]:
dataset = DEPRESJON_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [27]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)
#print(z_depresjon.shape)

In [28]:
print("Wyniki dla Depresjon:")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_depresjon.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Depresjon:
PART: day
  LR
    accuracy: 0.4685 +- 0.0607
    balanced_accuracy: 0.5103 +- 0.0398
    f1: 0.5443 +- 0.0182
    precision: 0.4307 +- 0.0367
    recall: 0.7636 +- 0.1078
    specificity: 0.2571 +- 0.1744
    ROC_AUC: 0.5103 +- 0.0398
    MCC: 0.0159 +- 0.0887

  SVM
    accuracy: 0.5824 +- 0.0012
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.0000 +- 0.0000
    precision: 1.0000 +- 0.0000
    recall: 0.0000 +- 0.0000
    specificity: 1.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  RF
    accuracy: 0.5713 +- 0.0335
    balanced_accuracy: 0.5229 +- 0.0265
    f1: 0.3026 +- 0.0605
    precision: 0.4956 +- 0.0987
    recall: 0.2289 +- 0.0674
    specificity: 0.8169 +- 0.0892
    ROC_AUC: 0.5229 +- 0.0265
    MCC: 0.0641 +- 0.0757

  LGBM
    accuracy: 0.6262 +- 0.0175
    balanced_accuracy: 0.5762 +- 0.0158
    f1: 0.3760 +- 0.0404
    precision: 0.6322 +- 0.0672
    recall: 0.2723 +- 0.0451
    specificity: 0.8800 +- 0.0478
    ROC_AU

# Classification - Psykose

In [29]:
dataset = PSYKOSE_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [30]:
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [31]:
print("Wyniki dla Psykose:")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_psykose.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Psykose:
PART: day
  LR
    accuracy: 0.4067 +- 0.0014
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.5782 +- 0.0014
    precision: 0.4067 +- 0.0014
    recall: 1.0000 +- 0.0000
    specificity: 0.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  SVM
    accuracy: 0.5933 +- 0.0014
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.0000 +- 0.0000
    precision: 1.0000 +- 0.0000
    recall: 0.0000 +- 0.0000
    specificity: 1.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  RF
    accuracy: 0.5785 +- 0.0147
    balanced_accuracy: 0.4935 +- 0.0192
    f1: 0.6611 +- 0.4174
    precision: 0.2000 +- 0.2449
    recall: 0.0377 +- 0.0555
    specificity: 0.9494 +- 0.0261
    ROC_AUC: 0.4935 +- 0.0192
    MCC: -0.0565 +- 0.0925

  LGBM
    accuracy: 0.6234 +- 0.0236
    balanced_accuracy: 0.5443 +- 0.0281
    f1: 0.1996 +- 0.0925
    precision: 0.7762 +- 0.1961
    recall: 0.1203 +- 0.0655
    specificity: 0.9683 +- 0.0284
    ROC_AUC

# Classification - Hyperactive

In [32]:
dataset = HYPERAKTIV_PREFIX
y_filename = "hyperaktiv_y.csv"

In [33]:
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [34]:
print("Wyniki dla Hyperaktiv:")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            #X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_psykose.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Hyperaktiv:
PART: day
  LR


/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3

    accuracy: 0.5570 +- 0.0249
    balanced_accuracy: 0.5747 +- 0.0209
    f1: 0.5706 +- 0.0345
    precision: 0.6864 +- 0.0180
    recall: 0.4889 +- 0.0416
    specificity: 0.6606 +- 0.0121
    ROC_AUC: 0.5747 +- 0.0209
    MCC: 0.1477 +- 0.0404

  SVM
    accuracy: 0.5632 +- 0.0522
    balanced_accuracy: 0.5510 +- 0.0532
    f1: 0.6266 +- 0.0521
    precision: 0.6475 +- 0.0414
    recall: 0.6111 +- 0.0786
    specificity: 0.4909 +- 0.0931
    ROC_AUC: 0.5510 +- 0.0532
    MCC: 0.1016 +- 0.1043

  RF
    accuracy: 0.5163 +- 0.0400
    balanced_accuracy: 0.5253 +- 0.0518
    f1: 0.5444 +- 0.0236
    precision: 0.6359 +- 0.0447
    recall: 0.4778 +- 0.0272
    specificity: 0.5727 +- 0.1110
    ROC_AUC: 0.5253 +- 0.0518
    MCC: 0.0504 +- 0.1019

  LGBM
    accuracy: 0.4094 +- 0.0131
    balanced_accuracy: 0.3995 +- 0.0166
    f1: 0.4756 +- 0.0248
    precision: 0.5125 +- 0.0154
    recall: 0.4444 +- 0.0351
    specificity: 0.3545 +- 0.0553
    ROC_AUC: 0.3995 +- 0.0166
    MCC: -0.1975 